# 1 - Predicting Salaries from Stack Overflow Surveys
Stack Overflow has been conducting [annual user surveys](https://insights.stackoverflow.com/survey/?utm_source=so-owned&utm_medium=blog&utm_campaign=dev-survey-2017&utm_content=blog-link&utm_term=data) starting in 2011. Yes, this is the same survey that (re)started the whole tabs vs spaces [debate](https://stackoverflow.blog/2017/06/15/developers-use-spaces-make-money-use-tabs/) in 2017. The results for the 2018 survey has been released, and I wanted to try **to use the 2017 results to try and predict salaries in the 2018 results**.  

For anyone who has worked on a dataset not from Kaggle or the UCI repository, you might have experienced of the 80/20 rule, where 80% of your time is spent cleaning data, and 20% on modeling.  Despite knowing the rule, it still surprised me how much time I spent cleaning the data, which is detailed below.

Broadly, I will be going through:

-  Downcasting data  
-  Identifying  and renaming common columns
-  Pre-processing data

#### 1.1 - Importing Libraries
Importing all standard libraries because its just habit now.  I've also set options to view up to 50 columns without truncation.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 50)

# 2 - Reading and Downcasting Data
Downcasting data means to optimize the datatype of each column to reduce memory usage.  For 2018, the dataset was more than 500 MB, which unfortunately is reaching the upper computational limits of my computer.  If you are interested in a more detailed explanation, check out my [kernel](https://www.kaggle.com/yscyang1/microsoft-malware-1-loading-a-large-data-set) for the Microsoft malware competition.

Both 2017 and 2018 had the same treatment.  First, I printed the breakdown of each datatype's memory usage, including the total memory usage.  Then I downcasted each column and checked to see that the downcasting occurred.  

Note:  I changed the columns "Respondent" from int32 to float32 because when saving to feather format, an error occurs with int32 dtype.

### 2.1 - 2017 Data
- Memory usage before downcasting: 405.03 MB
- Memory usage after downcasting: 15.56 MB
- About a 95% reduction in memory usage

In [3]:
df_2017 = pd.read_csv('2017/survey_results_public.csv')

In [4]:
df_2017.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51392 entries, 0 to 51391
Columns: 154 entries, Respondent to ExpectedSalary
dtypes: float64(6), int64(1), object(147)
memory usage: 405.0 MB


In [5]:
def get_memoryUsage(df):
    dtype_lst = list(df.get_dtype_counts().index)
    for dtype in dtype_lst:
        print('Total memory usage for {}: {} MB'.format(dtype, format(df.select_dtypes([dtype]).memory_usage(deep = True).sum()/1024**2,'.5f')))
    
    print('\n' + 'Total Memory Usage: {} MB'.format(format(df.memory_usage(deep=True).sum()/1024**2, '.2f')))

In [6]:
get_memoryUsage(df_2017)

Total memory usage for float64: 2.35262 MB
Total memory usage for int64: 0.39217 MB
Total memory usage for object: 402.28901 MB

Total Memory Usage: 405.03 MB


In [7]:
def downcast(df):
    for col in df.select_dtypes(['int64']):
        df[col] = pd.to_numeric(df[col], downcast = 'signed')
    for col in df.select_dtypes(['float64']):
        df[col] = pd.to_numeric(df[col], downcast = 'float')
    for col in df.select_dtypes(['object']):
        df[col] = df[col].astype('category')

In [8]:
downcast(df_2017)

In [9]:
get_memoryUsage(df_2017)

Total memory usage for category: 14.18455 MB
Total memory usage for float32: 1.17635 MB
Total memory usage for int32: 0.00008 MB

Total Memory Usage: 15.56 MB


In [10]:
df_2017['Respondent'] = df_2017['Respondent'].astype('float32')

### 2.2 - 2018 Data
- Memory usage before downcasting: 619.4 MB
- Memory usage after downcasting: 45.08 MB
- About a 90% reduction in memory usage

In [11]:
df_2018 = pd.read_csv('2018/survey_results_public.csv', low_memory=False)

In [12]:
get_memoryUsage(df_2018)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Columns: 129 entries, Respondent to SurveyEasy
dtypes: float64(41), int64(1), object(87)
memory usage: 619.4 MB


In [13]:
downcast(df_2018)

In [14]:
get_memoryUsage(df_2018)

Total memory usage for category: 29.23987 MB
Total memory usage for float32: 15.46125 MB
Total memory usage for int32: 0.00008 MB

Total Memory Usage: 45.08 MB


In [9]:
df_2018['Respondent'] = df_2018['Respondent'].astype('float32')

### 2.3 - A Brief Glance at the Columns
There are 154 columns in 2017 and 129 in 2018.  Yet, there are only 17 columns with the same name.  Surely there are more common columns between the two years?

In [15]:
pd.set_option('display.max_columns', 155)

In [16]:
df_2017.head(3)

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,InvestTimeTools,WorkPayCare,KinshipDevelopers,ChallengeMyself,CompetePeers,ChangeWorld,JobSeekingStatus,HoursPerWeek,LastNewJob,AssessJobIndustry,AssessJobRole,AssessJobExp,AssessJobDept,AssessJobTech,AssessJobProjects,AssessJobCompensation,AssessJobOffice,AssessJobCommute,AssessJobRemote,AssessJobLeaders,AssessJobProfDevel,AssessJobDiversity,AssessJobProduct,AssessJobFinances,ImportantBenefits,ClickyKeys,JobProfile,ResumePrompted,LearnedHiring,ImportantHiringAlgorithms,ImportantHiringTechExp,ImportantHiringCommunication,ImportantHiringOpenSource,ImportantHiringPMExp,ImportantHiringCompanies,ImportantHiringTitles,ImportantHiringEducation,ImportantHiringRep,ImportantHiringGettingThingsDone,Currency,Overpaid,TabsSpaces,EducationImportant,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,CousinEducation,WorkStart,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform,IDE,AuditoryEnvironment,Methodology,VersionControl,CheckInCode,ShipIt,OtherPeoplesCode,ProjectManagement,EnjoyDebugging,InTheZone,DifficultCommunication,CollaborateRemote,MetricAssess,EquipmentSatisfiedMonitors,EquipmentSatisfiedCPU,EquipmentSatisfiedRAM,EquipmentSatisfiedStorage,EquipmentSatisfiedRW,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1.0,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a soft ""g,"" like ""jiff""",Strongly agree,Strongly agree,Agree,Disagree,Strongly agree,Agree,Agree,Disagree,Somewhat agree,Disagree,Strongly agree,Strongly agree,Strongly disagree,Agree,Agree,Disagree,Agree,"I'm not actively looking, but I am open to new...",0.0,Not applicable/ never,Very important,Very important,Important,Very important,Very important,Very important,Important,Very important,Very important,Very important,Very important,Very important,Somewhat important,Not very important,Somewhat important,Stock options; Vacation/days off; Remote options,Yes,Other,NaN,NaN,Important,Important,Important,Somewhat important,Important,Not very important,Not very important,Not at all important,Somewhat important,Very important,NaN,NaN,Tabs,NaN,Online course; Open source contributions,NaN,NaN,NaN,6:00 AM,Swift,Swift,NaN,NaN,NaN,NaN,iOS,iOS,Atom; Xcode,Turn on some music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somewhat satisfied,Not very satisfied,Not at all satisfied,Very satisfied,Satisfied,Not very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have created a CV or Developer Story on Stac...,9.0,Desktop; iOS app,At le

In [17]:
df_2018.head(3)

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,Salary,SalaryType,ConvertedSalary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,IDE,OperatingSystem,NumberMonitors,Methodology,VersionControl,CheckInCode,AdBlocker,AdBlockerDisable,AdBlockerReasons,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1.0,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,3-5 years,3-5 years,Extremely satisfied,Extremely satisfied,Working as a founder or co-founder of my own c...,"I’m not actively looking, but I am open to new...",Less than a year ago,10.0,7.0,8.0,1.0,2.0,5.0,3.0,4.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,4.0,2.0,5.0,5.0,6.0,7.0,2.0,1.0,4.0,3.0,My job status or other personal status changed,NaN,NaN,Monthly,NaN,KES,Slack,One to three months,"Taught yourself a new language, framework, or ...",The official documentation and/or standards fo...,NaN,To build my professional network,Strongly agree,Strongly agree,Neither Agree nor Disagree,JavaScript;Python;HTML;CSS,JavaScript;Python;HTML;CSS,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,AWS;Azure;Linux;Firebase,AWS;Azure;Linux;Firebase,Django;React,Django;React,Komodo;Vim;Visual Studio Code,Linux-based,1,Agile;Scrum,Git,Multiple times per day,Yes,No,NaN,Strongly agree,Strongly agree,Strongly agree,Saw an online advertisement and then researche...,1.0,5.0,4.0,7.0,2.0,6.0,3.0,Artificial intelligence surpassing human intel...,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, and publicly",Upper management at the company/organization,Yes,10 (Very Likely),Multiple times per day,Yes,I have never participated in Q&A on Stack Over...,"No, I knew that Stack Overflow had a jobs boar...",Yes,NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Between 5:00 - 6:00 AM,9 - 12 hours,1 - 2 hours,Never,Standing desk,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy
1,3.0,Yes,Yes,U

In [18]:
print('Number of common columns: {} \n'.format(len(set(df_2017.columns).intersection(set(df_2018)))))
print(set(df_2017.columns).intersection(set(df_2018)))

Number of common columns: 17 

{'CheckInCode', 'Salary', 'CompanySize', 'Gender', 'VersionControl', 'SelfTaughtTypes', 'Respondent', 'FormalEducation', 'Currency', 'TimeAfterBootcamp', 'Methodology', 'CareerSatisfaction', 'LastNewJob', 'JobSatisfaction', 'EducationTypes', 'IDE', 'Country'}


### 2.4 - Identifying and Renaming Columns
From the documentation, I identified 49 columns in common between 2017 and 2018, including the 17 identified above.  I isolate each column and rename them to so both years have the same column names.

In [19]:
# Identifying columns
df_2017_keep = df_2017[['Respondent', 'ProgramHobby', 'Country', 'University', 'EmploymentStatus', 'FormalEducation', 
                        'MajorUndergrad', 'CompanySize', 'YearsProgram', 'YearsCodedJob', 'DeveloperType', 'CareerSatisfaction',
                       'JobSatisfaction', 'KinshipDevelopers', 'CompetePeers', 'LastNewJob', 'AssessJobIndustry', 'AssessJobDept',
                       'AssessJobTech', 'AssessJobCompensation', 'AssessJobOffice', 'AssessJobRemote', 'AssessJobProfDevel',
                       'AssessJobDiversity', 'AssessJobProduct', 'AssessJobFinances', 'ResumePrompted', 'Currency', 
                       'EducationTypes', 'SelfTaughtTypes', 'TimeAfterBootcamp', 'HaveWorkedLanguage', 'WantWorkLanguage',
                       'HaveWorkedFramework','WantWorkFramework', 'HaveWorkedDatabase', 'WantWorkDatabase', 'HaveWorkedPlatform',
                       'WantWorkPlatform', 'IDE', 'Methodology', 'VersionControl', 'CheckInCode', 'StackOverflowJobListing',
                       'Gender', 'HighestEducationParents', 'Race', 'SurveyLong', 'Salary']]

In [20]:
df_2018_keep = df_2018[['Respondent', 'Hobby', 'Country', 'Student', 'Employment', 'FormalEducation', 'UndergradMajor', 
                       'CompanySize', 'DevType', 'YearsCoding', 'YearsCodingProf', 'JobSatisfaction', 'CareerSatisfaction',
                       'LastNewJob', 'AssessJob1', 'AssessJob2', 'AssessJob3', 'AssessJob4', 'AssessJob5', 'AssessJob6',
                        'AssessJob7', 'AssessJob8', 'AssessJob9', 'AssessJob10', 'UpdateCV', 'Currency', 'ConvertedSalary',
                       'EducationTypes', 'SelfTaughtTypes', 'TimeAfterBootcamp', 'AgreeDisagree1', 'AgreeDisagree2',
                       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith', 'DatabaseDesireNextYear', 
                       'PlatformWorkedWith', 'PlatformDesireNextYear', 'FrameworkWorkedWith', 'FrameworkDesireNextYear',
                        'IDE', 'Methodology', 'VersionControl', 'CheckInCode', 'StackOverflowJobs', 'Gender', 
                        'EducationParents', 'RaceEthnicity', 'SurveyTooLong']]

In [21]:
# Renaming columns
df_2017_keep.rename(columns = {'Respondent': 'ID', 'ProgramHobby': 'Hobby', 'University': 'Student', 'EmploymentStatus': 'Employment',
                               'FormalEducation': 'Education', 'MajorUndergrad': 'UndergradMajor', 'YearsProgram': 'YearsCoding',
                              'YearsCodedJob': 'YearsCodingProf', 'DeveloperType': 'DevType', 'ResumePrompted': 'UpdateCV', 
                              'HaveWorkedLanguage': 'LanguageWorkedWith', 'WantWorkLanguage': 'LanguageDesireNextYear',
                              'HaveWorkedFramework': 'FrameworkWorkedWith', 'WantWorkFramework': 'FrameworkDesireNextYear',
                              'HaveWorkedDatabase': 'DatabaseWorkedWith', 'WantWorkDatabase': 'DatabaseDesireNextYear',
                              'HaveWorkedPlatform': 'PlatformWorkedWith', 'WantWorkPlatform': 'PlatformDesireNextYear', 
                              'StackOverflowJobListing': "StackOverflowJobs", 'HighestEducationParents': 'EducationParents'},
                   inplace = True)

C:\Users\Crystal\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [22]:
df_2018_keep.rename(columns = {'Respondent': 'ID', 'FormalEducation': 'Education', 'AssessJob1': 'AssessJobIndustry',
                               'AssessJob2': 'AssessJobFinances', 'AssessJob3': 'AssessJobDept', 'AssessJob4': 'AssessJobTech',
                               'AssessJob5': 'AssessJobCompensation', 'AssessJob6': 'AssessJobOffice', 
                               'AssessJob7': 'AssessJobRemote', 'AssessJob8': 'AssessJobProfDevel', 'AssessJob9': 'AssessJobDiversity',
                               'AssessJob10': 'AssessJobProduct', 'AgreeDisagree1': 'KinshipDevelopers', 'AgreeDisagree2': 'CompetePeers',
                              'RaceEthnicity': 'Race', 'SurveyTooLong': 'SurveyLong', 'ConvertedSalary': 'Salary'},
                   inplace = True)

C:\Users\Crystal\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


### 2.5 - Save to Feather
At this point, I would like to save my condensed raw data so I have something to go back to before I start manipulating things.

In [25]:
import os
os.makedirs('tmp', exist_ok=True)
df_2017_keep.to_feather('tmp/df_2017_1keep')
df_2018_keep.to_feather('tmp/df_2018_1keep')

# 3 - Processing Each Column
This is the last, but arguably the most important part of this post.

### 3.1 - Missing Data
Some respondents didn't fill out too much of the survey.  For example, one person filled out the hobby section, and left the rest blank.  Such answers are going to be useless for analysis, so I will drop all the rows htat have more than 50% of the answers blank.  This results in ~30% reduction of rows.

In [26]:
df_2017_keep.dropna(thresh=len(df_2017_keep.columns)/2, inplace=True)
df_2018_keep.dropna(thresh=len(df_2018_keep.columns)/2, inplace=True)

C:\Users\Crystal\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Crystal\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


### 3.2 - Salary
Since the main goal is to predict salary, any rows without a salary or currency is removed.  This results in removing about 66% and 35% of the rows in 2017 and 2018 respectively.  I haven't found in the documentation whether the salaries are already converted to US dollars in 2017, but working with the data, it seems like they are converted.  In the 2018 documentation, it clearly states salaries have been converted.

Since I want to know how much of each column is missing, I've written a function, getMissingPercent(), which takes a string of the column name and returns what percent of the column is missing for each year.  

In [27]:
def getMissingPercent(col):
    print('{} - percent missing in 2017: {}%'.format(col, df_2017_keep[col].isnull().sum()/len(df_2017_keep)*100))
    print('{} - percent missing in 2018: {}%'.format(col, df_2018_keep[col].isnull().sum()/len(df_2018_keep)*100))

In [28]:
getMissingPercent('Salary')

Salary - percent missing in 2017: 66.21510673234812%
Salary - percent missing in 2018: 34.16970486712783%


In [29]:
df_2017_keep = df_2017_keep[(df_2017_keep['Currency'].notnull()) & (df_2017_keep['Salary'].notnull())]
df_2018_keep = df_2018_keep[(df_2018_keep['Currency'].notnull()) & (df_2018_keep['Salary'].notnull())]

In [30]:
# Commented out in case need to convert 2017 currencies to USD

# currency_dict = {'British pounds sterling (£)': 1.27386, 'U.S. dollars ($)': 1, 'Euros (€)': 1.14630, 'Brazilian reais (R$)': 0.269293, 
#                  'Indian rupees (?)': 0.0142103, 'Polish zloty (zl)': 0.266836, 'Canadian dollars (C$)': 0.755728, 
#                  'Russian rubles (?)': 0.0148888, 'Swiss francs': 1.01940, 'Swedish kroner (SEK)': 0.112174, 
#                  'Mexican pesos (MXN$)': 0.0517878, 'Australian dollars (A$)': 0.715379, 'Japanese yen (¥)': 0.00917943, 
#                  'Chinese yuan renminbi (¥)': 0.146269, 'Singapore dollars (S$)': 0.736965, 'South African rands (R)': 0.0721070, 
#                  'Bitcoin (btc)': 4019.77}

In [31]:
# def convert_salary(col):
#     currency = col[0]
#     salary = col[1]

#     return currency_dict[currency] * salary

In [32]:
# df_2017_keep['Salary'] = df_2017_keep[['Currency','Salary']].apply(convert_salary, axis = 1)

### 3.3 - Hobby
Surprisingly, everyone filled out if they program as a hobby or not.  Although, in 2017, you could also answer if you contributed to open source projects whereas in 2018, answers constrained to yes or no.  I've simplified the 2017 answers so that anything aside from "no" becomes "yes".  

In [34]:
getMissingPercent('Hobby')

Hobby - percent missing in 2017: 0.0%
Hobby - percent missing in 2018: 0.0%


In [35]:
df_2017_keep['Hobby'].unique()

[Yes, both, Yes, I program as a hobby, No, Yes, I contribute to open source projects]
Categories (4, object): [Yes, both, Yes, I program as a hobby, No, Yes, I contribute to open source projects]

In [36]:
df_2018_keep['Hobby'].unique()

[Yes, No]
Categories (2, object): [Yes, No]

In [37]:
def hobby(col):
    for row in col:
        if row != 'No':
            return 'Yes'
        else:
            return 'No'

In [38]:
df_2017_keep['Hobby'] = df_2017_keep[['Hobby']].apply(hobby, axis = 1)

### 3.4 - Country
Respondents state what country they reside in.  Again, no missing data.  But respondents had to type in the country name, so watch out for typos later.

In [39]:
getMissingPercent('Country')

Country - percent missing in 2017: 0.0%
Country - percent missing in 2018: 0.0%


### 3.5 - Student
For both years, this asks if the respondent is currently enrolled in a college or university program.  They have the same answer choices of full time, part time, no, or prefer not to say.  However, in the 2018 dataset, "I prefer not to say" all comes out as null values.  This seems to be true for all of 2018 data.  Null values are filled in with "I prefer not to say".

In [40]:
getMissingPercent('Student')

Student - percent missing in 2017: 0.0%
Student - percent missing in 2018: 0.7303854295085017%


In [41]:
df_2017_keep['Student'].unique()

[No, Yes, part-time, Yes, full-time, I prefer not to say]
Categories (4, object): [No, Yes, part-time, Yes, full-time, I prefer not to say]

In [42]:
df_2018_keep['Student'].unique()

[No, Yes, part-time, Yes, full-time, NaN]
Categories (3, object): [No, Yes, part-time, Yes, full-time]

In [43]:
df_2018_keep['Student'] = df_2018_keep.Student.cat.add_categories('I prefer not to say').fillna('I prefer not to say')

### 3.6 - Employment
After removing null salaries, 2017 only has two employment statuses: employed full time or part time.  In the 2017 documentation, it states that salary information was only collected if respondents stated they were employed.  

There was no such filter for the 2018 data, so I've filtered out anyone unemployed, which includes independent contractors/freelancers/self-employed, those unemployed but looking for work, those unemployed and not looking for work, and retired people.

In [44]:
getMissingPercent('Employment')

Employment - percent missing in 2017: 0.0%
Employment - percent missing in 2018: 0.2907585308552953%


In [45]:
sorted(df_2017_keep['Employment'].unique())

['Employed full-time', 'Employed part-time']

In [46]:
df_2018_keep['Employment'].unique()

[Employed full-time, Employed part-time, Independent contractor, freelancer, or self-em..., Not employed, and not looking for work, NaN, Not employed, but looking for work, Retired]
Categories (6, object): [Employed full-time, Employed part-time, Independent contractor, freelancer, or self-em..., Not employed, and not looking for work, Not employed, but looking for work, Retired]

In [47]:
df_2018_keep = df_2018_keep[(df_2018_keep['Employment']=='Employed full-time') | (df_2018_keep['Employment']=='Employed part-time')]

### 3.7 - Education
Education refers to the highest level of formal education that the respondent has completed.  In 2018, a category was added for associates degree.  I've added that to 2017 categories and converted null values in 2018 to "I prefer not to say".

In [48]:
getMissingPercent('Education')

Education - percent missing in 2017: 0.0%
Education - percent missing in 2018: 1.3720120183998512%


In [49]:
list(df_2017_keep['Education'].unique())

["Bachelor's degree",
 'Professional degree',
 "Master's degree",
 "Some college/university study without earning a bachelor's degree",
 'Doctoral degree',
 'Secondary school',
 'I never completed any formal education',
 'Primary/elementary school',
 'I prefer not to answer']

In [50]:
list(df_2018_keep['Education'].unique())

['Bachelor’s degree (BA, BS, B.Eng., etc.)',
 'Some college/university study without earning a degree',
 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
 'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
 'Associate degree',
 'Professional degree (JD, MD, etc.)',
 'Other doctoral degree (Ph.D, Ed.D., etc.)',
 nan,
 'Primary/elementary school',
 'I never completed any formal education']

In [51]:
df_2017_keep['Education'] = df_2017_keep.Education.cat.add_categories('Associate degree')
df_2018_keep['Education'] = df_2018_keep.Education.cat.add_categories('I prefer not to answer').fillna('I prefer not to answer')

### 3.8 - Undergraduate Major
As one would expect this column asks what is/was the respondent's undergraduate major.  The two years have the same options to choose from, and it encompasses a wide variety of majors, with heavy emphasis on different types of computer science majors.

In [52]:
getMissingPercent('UndergradMajor')

UndergradMajor - percent missing in 2017: 6.91035683202785%
UndergradMajor - percent missing in 2018: 9.447206785609827%


In [53]:
list(df_2017_keep['UndergradMajor'].unique())==list(df_2017_keep['UndergradMajor'].unique())

True

In [54]:
df_2017_keep['UndergradMajor'] = df_2017_keep.UndergradMajor.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['UndergradMajor'] = df_2018_keep.UndergradMajor.cat.add_categories('NaN').fillna('NaN')

### 3.9 - Company Size
Company size options range from fewer than 10 employees to greater than 10k employees.

In [55]:
getMissingPercent('CompanySize')

CompanySize - percent missing in 2017: 0.08703220191470844%
CompanySize - percent missing in 2018: 1.59536281209285%


In [56]:
df_2017_keep['CompanySize'] = df_2017_keep.CompanySize.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['CompanySize'] = df_2018_keep.CompanySize.cat.add_categories('NaN').fillna('NaN')

In [57]:
sorted(df_2017_keep['UndergradMajor'].unique())==sorted(df_2017_keep['UndergradMajor'].unique())

True

### 3.10 - Years Coded 
This section asks how many years the respondent has been coding, including for school, for fun, or for work (professionally).  The answer choices for 2017 were a little confusing.  For example, answer choices included 1 to 2 years, 2 to 3 years, and so forth.  2018 choices were less ambiguous, with example choices of 0-2 years and 3-5 years.  

For the 2017 choices, I've reworked the answer choices so that the first number is included, and the second is excluded.  To clarify, if the respondent chose answer choice 1 to to years, it means they have been coding anywhere between 1 to 1.99 years.  With this method, I am able to make the same answer choices between the two datasets.

In [58]:
getMissingPercent('YearsCoding')

YearsCoding - percent missing in 2017: 0.06092254134029591%
YearsCoding - percent missing in 2018: 0.034566194262011754%


In [59]:
df_2017_keep['YearsCoding'] = df_2017_keep.YearsCoding.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['YearsCoding'] = df_2018_keep.YearsCoding.cat.add_categories('NaN').fillna('NaN')

In [60]:
YearsCoding2017_dict ={'Less than a year':  '0-2 years',   '1 to 2 years': '0-2 years', '2 to 3 years':  '3-5 years',
                      '3 to 4 years':  '3-5 years', '4 to 5 years': '3-5 years', '5 to 6 years':  '6-8 years',
                       '6 to 7 years': '6-8 years', '7 to 8 years': '6-8 years', '8 to 9 years':  '9-11 years',
                      '9 to 10 years':  '9-11 years', '10 to 11 years': '9-11 years', '11 to 12 years':  '12-14 years',
                      '12 to 13 years': '12-14 years', '13 to 14 years': '12-14 years', '14 to 15 years':  '15-17 years',
                      '15 to 16 years':  '15-17 years', '16 to 17 years': '15-17 years', '17 to 18 years':  '18-20 years',
                      '18 to 19 years': '18-20 years', '19 to 20 years': '18-20 years', '20 or more years': '20 or more years',
                      'NaN': 'NaN'}

In [61]:
def convert_YearsCoding2017(col):
    return YearsCoding2017_dict[col]

In [62]:
df_2017_keep['YearsCoding'] = df_2017_keep['YearsCoding'].apply(convert_YearsCoding2017)

In [63]:
YearsCoding2018_dict = {'21-23 years': '20 or more years', '24-26 years': '20 or more years', '27-29 years': '20 or more years',
                       '30 or more years': '20 or more years', 'NaN': 'NaN'}

In [64]:
def convert_YearsCoding2018(col):
    try:
        return YearsCoding2018_dict[col]
    except:
        return col

In [65]:
df_2018_keep['YearsCoding'] = df_2018_keep['YearsCoding'].apply(convert_YearsCoding2018)

### 3.11 - Years Coded Professionally
Similar to section 3.10's Years Coded, but only applies to the years that the respondent has coded for work.  I was able to reuse the years coding dictionary.

In [66]:
getMissingPercent('YearsCodingProf')

YearsCodingProf - percent missing in 2017: 0.38294168842471715%
YearsCodingProf - percent missing in 2018: 0.8668137945704486%


In [67]:
df_2017_keep['YearsCodingProf'] = df_2017_keep.YearsCodingProf.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['YearsCodingProf'] = df_2018_keep.YearsCodingProf.cat.add_categories('NaN').fillna('NaN')

In [68]:
df_2017_keep['YearsCodingProf'] = df_2017_keep['YearsCodingProf'].apply(convert_YearsCoding2017)
df_2018_keep['YearsCodingProf'] = df_2018_keep['YearsCodingProf'].apply(convert_YearsCoding2018)

### 3.12 -  Software Developer Type
This question asks the respondent what type of software developer they are.  Multiple responses are allowed, which has resulted in ~900 and 4800 unique responses for 2017 and 2018 respectively.  For now, I will fill in the null values as "NaN", and create a new column that indicates how many options the respondent chose, as written in get_count().

In [69]:
getMissingPercent('DevType')

DevType - percent missing in 2017: 0.017406440382941687%
DevType - percent missing in 2018: 0.7551383977239491%


In [70]:
df_2017_keep['DevType']= df_2017_keep.DevType.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['DevType']= df_2018_keep.DevType.cat.add_categories('NaN').fillna('NaN')

In [71]:
len(df_2017_keep['DevType'][36].split(";"))

3

In [72]:
def get_count(col):
    count = len(col.split(';'))
    if col == 'NaN':
        count = 0
    return count

In [73]:
df_2017_keep['DevType_Count'] = df_2017_keep['DevType'].apply(get_count)

In [74]:
df_2018_keep['DevType_Count'] = df_2018_keep['DevType'].apply(get_count)

### 3.13 - Career Satisfaction
This question asks responded how satisfied they are with their career so far.  Again, between 2017 and 2018, completely different answer systems are used, where 2017 uses a 0 to 10 scale (where 0 is most dissatisfied and 10 is most satisfied), and in 2018, answers range from extremely dissatisfied to extremely satisfied. To combine the two, I've anchored it so that a 0 correlates to extremely dissatisfied, 5 to neither satisfied nor dissatisfied, and 10 to extremely satisfied.

In [75]:
getMissingPercent('CareerSatisfaction')

CareerSatisfaction - percent missing in 2017: 0.20017406440382943%
CareerSatisfaction - percent missing in 2018: 0.8668137945704486%


In [76]:
list(df_2017_keep['CareerSatisfaction'].unique())

[8.0, 9.0, 7.0, 10.0, nan, 6.0, 5.0, 4.0, 3.0, 2.0, 0.0, 1.0]

In [77]:
list(df_2018_keep['CareerSatisfaction'].unique())

['Neither satisfied nor dissatisfied',
 'Moderately satisfied',
 'Slightly satisfied',
 'Moderately dissatisfied',
 'Slightly dissatisfied',
 'Extremely satisfied',
 'Extremely dissatisfied',
 nan]

In [78]:
satisfaction_dict = {0.0: 'Extremely dissatisfied', 1.0:  'Moderately dissatisfied', 2.0:  'Moderately dissatisfied',
                    3.0: 'Slightly dissatisfied', 4.0: 'Slightly dissatisfied', 5.0: 'Neither satisfied nor dissatisfied',
                    6.0:  'Slightly satisfied', 7.0:  'Slightly satisfied', 8.0:  'Moderately satisfied', 9.0:  'Moderately satisfied',
                    10.0:  'Extremely satisfied', 'NaN': 'NaN'}

In [79]:
def convert_satisfaction(col):
    return satisfaction_dict[col]

In [80]:
df_2017_keep['CareerSatisfaction'] = df_2017_keep['CareerSatisfaction'].astype('category')
df_2017_keep['CareerSatisfaction'] = df_2017_keep.CareerSatisfaction.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['CareerSatisfaction'] = df_2018_keep.CareerSatisfaction.cat.add_categories('NaN').fillna('NaN')

df_2017_keep['CareerSatisfaction'] = df_2017_keep['CareerSatisfaction'].apply(convert_satisfaction)

### 3.14 - Job Satisfaction
This question is similar to section 3.13, but specific to the respondent's current job.  Data processing is also similar to career satisfaction.

In [81]:
getMissingPercent('JobSatisfaction')

JobSatisfaction - percent missing in 2017: 0.26979982593559615%
JobSatisfaction - percent missing in 2018: 0.9625355632960195%


In [82]:
df_2017_keep['JobSatisfaction'].unique()

array([ 9.,  8.,  7., nan,  5.,  6.,  2., 10.,  4.,  3.,  1.,  0.])

In [83]:
df_2018_keep['JobSatisfaction'].unique()

[Moderately dissatisfied, Slightly satisfied, Moderately satisfied, Neither satisfied nor dissatisfied, Slightly dissatisfied, Extremely dissatisfied, NaN, Extremely satisfied]
Categories (7, object): [Moderately dissatisfied, Slightly satisfied, Moderately satisfied, Neither satisfied nor dissatisfied, Slightly dissatisfied, Extremely dissatisfied, Extremely satisfied]

In [84]:
df_2017_keep['JobSatisfaction'] = df_2017_keep['JobSatisfaction'].astype('category')
df_2017_keep['JobSatisfaction'] = df_2017_keep.JobSatisfaction.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['JobSatisfaction'] = df_2018_keep.JobSatisfaction.cat.add_categories('NaN').fillna('NaN')

df_2017_keep['JobSatisfaction'] = df_2017_keep['JobSatisfaction'].apply(convert_satisfaction)

### 3.15 - Kinship with Developers
Kinship refers to the sense of connection the respondent feels with other developers.  The two years have the same 5 point scale ranging from strongly disagree to strongly agree.  A minor difference is 2017's somewhat agree is equivalent to 2018's neither agree nor disagree.  I've decided to change this to a numerical scale where strongly disagree is a 1 and strongly agree is a 5.

In [85]:
getMissingPercent('KinshipDevelopers')

KinshipDevelopers - percent missing in 2017: 30.139251523063532%
KinshipDevelopers - percent missing in 2018: 2.3292297056555613%


In [86]:
list(df_2017_keep['KinshipDevelopers'].unique())

['Somewhat agree',
 'Agree',
 nan,
 'Strongly agree',
 'Disagree',
 'Strongly disagree']

In [87]:
list(df_2018_keep['KinshipDevelopers'].unique())

['Agree',
 'Strongly agree',
 'Disagree',
 'Neither Agree nor Disagree',
 nan,
 'Strongly disagree']

In [88]:
agree_dict = {'Strongly disagree': 1, 'Disagree': 2, 'Somewhat agree': 3, 'Neither Agree nor Disagree': 3, 'Agree': 4, 
             'Strongly agree': 5}

In [89]:
def convert_agreement(col):
    return agree_dict[col]

In [90]:
df_2017_keep['KinshipDevelopers'] = df_2017_keep['KinshipDevelopers'].apply(convert_agreement)
df_2018_keep['KinshipDevelopers'] = df_2018_keep['KinshipDevelopers'].apply(convert_agreement)

In [91]:
df_2017_keep['KinshipDevelopers'] = pd.to_numeric(df_2017_keep['KinshipDevelopers'], downcast='unsigned')
df_2018_keep['KinshipDevelopers'] = pd.to_numeric(df_2018_keep['KinshipDevelopers'], downcast='unsigned')

C:\Users\Crystal\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [92]:
df_2017_keep['KinshipDevelopers'] = df_2017_keep['KinshipDevelopers'].fillna(0)
df_2018_keep['KinshipDevelopers'] = df_2018_keep['KinshipDevelopers'].fillna(0)

### 3.16 - Compete with Peers
For the philosophers, would this be the opposite or similar emotion to kinship with other developers (section 3.15)?  As the title suggests, the survey is questioning if the respondent thinks of themselves in competition with their peers.  It uses the same 5 point scale as section 3.15. 

In [93]:
getMissingPercent('CompetePeers')

CompetePeers - percent missing in 2017: 29.930374238468232%
CompetePeers - percent missing in 2018: 2.222872184849371%


In [94]:
df_2017_keep['CompetePeers'].unique()

[Disagree, Somewhat agree, Strongly disagree, NaN, Agree, Strongly agree]
Categories (5, object): [Disagree, Somewhat agree, Strongly disagree, Agree, Strongly agree]

In [95]:
df_2018_keep['CompetePeers'].unique()

[Agree, Neither Agree nor Disagree, Strongly disagree, Disagree, NaN, Strongly agree]
Categories (5, object): [Agree, Neither Agree nor Disagree, Strongly disagree, Disagree, Strongly agree]

In [96]:
df_2017_keep['CompetePeers'] = df_2017_keep['CompetePeers'].apply(convert_agreement)
df_2018_keep['CompetePeers'] = df_2018_keep['CompetePeers'].apply(convert_agreement)

In [97]:
df_2017_keep['CompetePeers'] = pd.to_numeric(df_2017_keep['CompetePeers'], downcast='unsigned')
df_2018_keep['CompetePeers'] = pd.to_numeric(df_2018_keep['CompetePeers'], downcast='unsigned')

C:\Users\Crystal\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [98]:
df_2017_keep['CompetePeers'] = df_2017_keep['CompetePeers'].fillna(0)
df_2018_keep['CompetePeers'] = df_2018_keep['CompetePeers'].fillna(0)

### 3.17 - Last New Job
This section asks when was the last time the respondent took a job with a new employer.  Responses range from never to more than four years ago.  I changed 2018's 'I've never had a job' to 'Not applicable/ never' to match 2017's response.

In [99]:
getMissingPercent('LastNewJob')

LastNewJob - percent missing in 2017: 28.006962576153178%
LastNewJob - percent missing in 2018: 0.1435826530883565%


In [100]:
list(df_2017_keep['LastNewJob'].unique())

[nan,
 'More than 4 years ago',
 'Less than a year ago',
 'Between 1 and 2 years ago',
 'Between 2 and 4 years ago',
 'Not applicable/ never']

In [101]:
list(df_2018_keep['LastNewJob'].unique())

['More than 4 years ago',
 'Between 1 and 2 years ago',
 'Between 2 and 4 years ago',
 'Less than a year ago',
 nan,
 "I've never had a job"]

In [102]:
df_2017_keep['LastNewJob'] = df_2017_keep.LastNewJob.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['LastNewJob'] = df_2018_keep.LastNewJob.cat.add_categories('NaN').fillna('NaN')

In [103]:
df_2018_keep['LastNewJob'] = df_2018_keep['LastNewJob'].replace("I've never had a job", 'Not applicable/ never')

### 3.18 - Assessing Jobs: Industry
The subsequent assessing jobs sections are based on if the respondent is assessing a potential job to apply to, how important is each category.  In this section, the category is the industry.  

For all of the assessing jobs columns, 2017 potential responses range from not at all important to very important, whereas 2018's responses range from 1 to 10.  I've anchored it so that a 1 corresponds to not at all important, 5 is somewhat important, and 10 corresponds to very important.

In [104]:
getMissingPercent('AssessJobIndustry')

AssessJobIndustry - percent missing in 2017: 47.136640557006096%
AssessJobIndustry - percent missing in 2018: 3.866095881305007%


In [105]:
list(df_2017_keep['AssessJobIndustry'].unique())

[nan,
 'Important',
 'Not very important',
 'Very important',
 'Somewhat important',
 'Not at all important']

In [106]:
list(df_2018_keep['AssessJobIndustry'].unique())

[1.0, 8.0, 5.0, 6.0, 9.0, 7.0, 3.0, 10.0, nan, 4.0, 2.0]

In [107]:
df_2018_keep['AssessJobIndustry'] = df_2018_keep['AssessJobIndustry'].astype('category')

In [108]:
df_2018_keep['AssessJobIndustry'] = df_2018_keep.AssessJobIndustry.cat.add_categories('NaN').fillna('NaN')

In [109]:
df_2017_keep['AssessJobIndustry'] = df_2017_keep.AssessJobIndustry.cat.add_categories('NaN').fillna('NaN')

In [110]:
importance_dict = {1: 'Not at all important', 2: 'Not at all important', 3:  'Not very important', 4:  'Not very important',
                  5:  'Somewhat important', 6:  'Somewhat important', 7:  'Important', 8:  'Important', 9:  'Very important',
                  10:  'Very important', 'NaN': 'NaN'}

In [111]:
def convert_importance(col):
    return importance_dict[col]

In [112]:
df_2018_keep['AssessJobIndustry'] = df_2018_keep['AssessJobIndustry'].apply(convert_importance)

### 3.19 - Assessing Jobs: Department
How important is the specific team or department when assessing potential jobs?  

In [113]:
getMissingPercent('AssessJobDept')

AssessJobDept - percent missing in 2017: 47.17145343777197%
AssessJobDept - percent missing in 2018: 3.866095881305007%


In [114]:
list(df_2017_keep['AssessJobDept'].unique())

[nan,
 'Very important',
 'Not very important',
 'Important',
 'Somewhat important',
 'Not at all important']

In [115]:
df_2018_keep['AssessJobDept'].unique()

array([10.,  7.,  4.,  9.,  2., nan,  6.,  8.,  5.,  3.,  1.])

In [116]:
df_2018_keep['AssessJobDept'] = df_2018_keep['AssessJobDept'].astype('category')

In [117]:
df_2017_keep['AssessJobDept'] = df_2017_keep.AssessJobDept.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobDept'] = df_2018_keep.AssessJobDept.cat.add_categories('NaN').fillna('NaN')

In [118]:
df_2018_keep['AssessJobDept'] = df_2018_keep['AssessJobDept'].apply(convert_importance)

### 3.20 - Assessing Jobs: Technology
How important is the language, frameworks, and/or other technologies when assessing a potential job?

In [119]:
getMissingPercent('AssessJobTech')

AssessJobTech - percent missing in 2017: 47.06701479547433%
AssessJobTech - percent missing in 2018: 3.866095881305007%


In [120]:
df_2018_keep['AssessJobTech'] = df_2018_keep['AssessJobTech'].astype('category')

In [121]:
df_2017_keep['AssessJobTech'] = df_2017_keep.AssessJobTech.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobTech'] = df_2018_keep.AssessJobTech.cat.add_categories('NaN').fillna('NaN')

In [122]:
df_2018_keep['AssessJobTech'] = df_2018_keep['AssessJobTech'].apply(convert_importance)

### 3.21 - Assessing Jobs: Compensation
How important are the benefits and compensation when assessing a potential job?

In [123]:
getMissingPercent('AssessJobCompensation')

AssessJobCompensation - percent missing in 2017: 47.11923411662315%
AssessJobCompensation - percent missing in 2018: 3.866095881305007%


In [124]:
df_2018_keep['AssessJobCompensation'] = df_2018_keep['AssessJobCompensation'].astype('category')

In [125]:
df_2017_keep['AssessJobCompensation'] = df_2017_keep.AssessJobCompensation.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobCompensation'] = df_2018_keep.AssessJobCompensation.cat.add_categories('NaN').fillna('NaN')

In [126]:
df_2018_keep['AssessJobCompensation'] = df_2018_keep['AssessJobCompensation'].apply(convert_importance)

### 3.22 -  Assessing Jobs: Office
How important is the office environment/company culture when assessing a potential job?

In [127]:
getMissingPercent('AssessJobOffice')

AssessJobOffice - percent missing in 2017: 47.180156657963444%
AssessJobOffice - percent missing in 2018: 3.866095881305007%


In [128]:
df_2018_keep['AssessJobOffice'] = df_2018_keep['AssessJobOffice'].astype('category')

In [129]:
df_2017_keep['AssessJobOffice'] = df_2017_keep.AssessJobOffice.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobOffice'] = df_2018_keep.AssessJobOffice.cat.add_categories('NaN').fillna('NaN')

In [130]:
df_2018_keep['AssessJobOffice'] = df_2018_keep['AssessJobOffice'].apply(convert_importance)

### 3.23 - Assessing Jobs: Work Remotely
How important is working from home/remotely when assessing a potential job?

In [131]:
getMissingPercent("AssessJobRemote")

AssessJobRemote - percent missing in 2017: 47.14534377719756%
AssessJobRemote - percent missing in 2018: 3.866095881305007%


In [132]:
df_2018_keep['AssessJobRemote'] = df_2018_keep['AssessJobRemote'].astype('category')

In [133]:
df_2017_keep['AssessJobRemote'] = df_2017_keep.AssessJobRemote.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobRemote'] = df_2018_keep.AssessJobRemote.cat.add_categories('NaN').fillna('NaN')

In [134]:
df_2018_keep['AssessJobRemote'] = df_2018_keep['AssessJobRemote'].apply(convert_importance)

### 3.24 - Assessing Jobs: Professional Development
How important are opportunities for professional development when assessing a potential job?

In [135]:
getMissingPercent('AssessJobProfDevel')

AssessJobProfDevel - percent missing in 2017: 47.127937336814625%
AssessJobProfDevel - percent missing in 2018: 3.866095881305007%


In [136]:
df_2018_keep['AssessJobProfDevel'] = df_2018_keep['AssessJobProfDevel'].astype('category')

In [137]:
df_2017_keep['AssessJobProfDevel'] = df_2017_keep.AssessJobProfDevel.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobProfDevel'] = df_2018_keep.AssessJobProfDevel.cat.add_categories('NaN').fillna('NaN')

In [138]:
df_2018_keep['AssessJobProfDevel'] = df_2018_keep['AssessJobProfDevel'].apply(convert_importance)

### 3.25 - Assessing Jobs: Diversity
How important is the diversity of the company or organization when assessing a potential job?

In [139]:
getMissingPercent('AssessJobDiversity')

AssessJobDiversity - percent missing in 2017: 47.05831157528286%
AssessJobDiversity - percent missing in 2018: 3.866095881305007%


In [140]:
df_2018_keep['AssessJobDiversity'] = df_2018_keep['AssessJobDiversity'].astype('category')

In [141]:
df_2017_keep['AssessJobDiversity'] = df_2017_keep.AssessJobDiversity.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobDiversity'] = df_2018_keep.AssessJobDiversity.cat.add_categories('NaN').fillna('NaN')

In [142]:
df_2018_keep['AssessJobDiversity'] = df_2018_keep['AssessJobDiversity'].apply(convert_importance)

### 3.26 - Assessing Jobs: Product Impact
How important is the impactfulness of the product or service the respondent would be working on when assessing a potential job?

In [143]:
getMissingPercent('AssessJobProduct')

AssessJobProduct - percent missing in 2017: 47.136640557006096%
AssessJobProduct - percent missing in 2018: 3.866095881305007%


In [144]:
df_2018_keep['AssessJobProduct'] = df_2018_keep['AssessJobProduct'].astype('category')

In [145]:
df_2017_keep['AssessJobProduct'] = df_2017_keep.AssessJobProduct.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobProduct'] = df_2018_keep.AssessJobProduct.cat.add_categories('NaN').fillna('NaN')

In [146]:
df_2018_keep['AssessJobProduct'] = df_2018_keep['AssessJobProduct'].apply(convert_importance)

### 3.27 - Assessing Jobs: Finances
How important is the financial performance or funding status of the company when assessing a potential job?

In [147]:
getMissingPercent('AssessJobFinances')

AssessJobFinances - percent missing in 2017: 47.10182767624021%
AssessJobFinances - percent missing in 2018: 3.866095881305007%


In [148]:
df_2018_keep['AssessJobFinances'] = df_2018_keep['AssessJobFinances'].astype('category')

In [149]:
df_2017_keep['AssessJobFinances'] = df_2017_keep.AssessJobFinances.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['AssessJobFinances'] = df_2018_keep.AssessJobFinances.cat.add_categories('NaN').fillna('NaN')

In [150]:
df_2018_keep['AssessJobFinances'] = df_2018_keep['AssessJobFinances'].apply(convert_importance)

### 3.28 - Reason for Updated CV
What was the reason the respondent last updated their resume/CV?  They could only pick one response, but between the two years, the responses were vastly different.  I added categories as appropriate to each year.

In [151]:
getMissingPercent('UpdateCV')

UpdateCV - percent missing in 2017: 30.513489991296776%
UpdateCV - percent missing in 2018: 3.6454040256321623%


In [152]:
list(df_2017_keep['UpdateCV'].unique())

[nan,
 'A recruiter contacted me',
 'A friend told me about a job opportunity',
 'I completed a major project, assignment, or contract',
 'Something else',
 "I saw an employer's advertisement",
 'I was just giving it a regular update',
 'I received bad news about the future of my company or depart',
 'I received negative feedback on my job performance']

In [153]:
list(df_2018_keep['UpdateCV'].unique())

['I saw an employer’s advertisement',
 'My job status or other personal status changed',
 'I did not receive an expected change in compensation',
 'A friend told me about a job opportunity',
 'I had a negative experience or interaction at work',
 nan,
 'A recruiter contacted me',
 'I received bad news about the future of my company or department',
 'I received negative feedback on my job performance']

In [154]:
df_2017_keep['UpdateCV'] = df_2017_keep.UpdateCV.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['UpdateCV'] = df_2018_keep.UpdateCV.cat.add_categories('NaN').fillna('NaN')

In [155]:
df_2017_keep['UpdateCV'] = df_2017_keep.UpdateCV.cat.add_categories(['My job status or other personal status changed',
                                                                     'I did not receive an expected change in compensation',
                                                                     'I had a negative experience or interaction at work',
                                                                     'I received bad news about the future of my company or department'])

In [156]:
df_2018_keep['UpdateCV'] = df_2018_keep.UpdateCV.cat.add_categories(['I completed a major project, assignment, or contract',
                                                                     'Something else',
                                                                     'I was just giving it a regular update'])

### 3.29 -  Informal Schooling Education Types
The respondent is asked what types of activities they have participated in outside of their formal schooling.  Multiple answers are allowed, resulting in ~425 unique responses for each year.  Answers include anything from taking an online programming course, participating in coding competitions or hackathons, and contributing to open source software.

In [157]:
getMissingPercent('EducationTypes')

EducationTypes - percent missing in 2017: 29.89556135770235%
EducationTypes - percent missing in 2018: 3.4965034965034967%


In [158]:
df_2017_keep['EducationTypes'].unique()

[Self-taught; Coding competition; Hackathon; Op..., NaN, On-the-job training; Self-taught; Coding compe..., Self-taught, Part-time/evening course; Self-taught; Coding ..., ..., Online course; Bootcamp; Part-time/evening cou..., Bootcamp; On-the-job training; Open source con..., Online course; Industry certification; Coding ..., Online course; Bootcamp; Part-time/evening cou..., Online course; Bootcamp; Part-time/evening course]
Length: 348
Categories (347, object): [Self-taught; Coding competition; Hackathon; Op..., On-the-job training; Self-taught; Coding compe..., Self-taught, Part-time/evening course; Self-taught; Coding ..., ..., Bootcamp; On-the-job training; Open source con..., Online course; Industry certification; Coding ..., Online course; Bootcamp; Part-time/evening cou..., Online course; Bootcamp; Part-time/evening course]

In [159]:
df_2017_keep['EducationTypes']= df_2017_keep.EducationTypes.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['EducationTypes']= df_2018_keep.EducationTypes.cat.add_categories('NaN').fillna('NaN')

In [160]:
df_2017_keep['EducationTypes_Count'] = df_2017_keep['EducationTypes'].apply(get_count)
df_2018_keep['EducationTypes_Count'] = df_2018_keep['EducationTypes'].apply(get_count)

### 3.30 - Resources for the Self Taught 
Respondents who indicated they taught themselves a programming technology without taking a course are asked what resources they went to.  Sources include books, Stack Overflow, and official documentation.

In [161]:
getMissingPercent('SelfTaughtTypes')

SelfTaughtTypes - percent missing in 2017: 35.38729329852045%
SelfTaughtTypes - percent missing in 2018: 14.456646015581375%


In [162]:
df_2017_keep['SelfTaughtTypes']= df_2017_keep.SelfTaughtTypes.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['SelfTaughtTypes']= df_2018_keep.SelfTaughtTypes.cat.add_categories('NaN').fillna('NaN')

### 3.31 - Time After Bootcamp to get Hired
For respondents who indicated they went to a bootcamp, this question asks how long did it take for each person to get hired after the camp. Both years have essentially the same options, but the wording is slightly different.  

Also note that there is an extremely high number of missing data for both years.

In [163]:
getMissingPercent('TimeAfterBootcamp')

TimeAfterBootcamp - percent missing in 2017: 94.02088772845954%
TimeAfterBootcamp - percent missing in 2018: 90.71498843361961%


In [164]:
list(df_2017_keep['TimeAfterBootcamp'].unique())

[nan,
 'I already had a job as a developer when I started the program',
 'One to three months',
 'I got a job as a developer before completing the program',
 'Less than a month',
 'Four to six months',
 'Immediately upon graduating',
 'Longer than a year',
 'Six months to a year',
 "I haven't gotten a job as a developer yet"]

In [165]:
list(df_2018_keep['TimeAfterBootcamp'].unique())

[nan,
 'I already had a full-time job as a developer when I began the program',
 'Immediately after graduating',
 'Four to six months',
 'One to three months',
 'I haven’t gotten a developer job',
 'Six months to a year',
 'Less than a month',
 'Longer than a year']

In [166]:
df_2017_keep['TimeAfterBootcamp']= df_2017_keep.TimeAfterBootcamp.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['TimeAfterBootcamp']= df_2018_keep.TimeAfterBootcamp.cat.add_categories('NaN').fillna('NaN')

In [167]:
def convert_TimeAfterBootcamp(col):
    if col ==  'I already had a full-time job as a developer when I began the program':
        return  'I already had a job as a developer when I started the program'
    elif col ==  'Immediately after graduating':
        return  'Immediately upon graduating'
    elif col ==  'I haven’t gotten a developer job':
        return  "I haven't gotten a job as a developer yet"
    else:
        return col

In [168]:
df_2018_keep['TimeAfterBootcamp'] = df_2018_keep['TimeAfterBootcamp'].apply(convert_TimeAfterBootcamp)

In [169]:
df_2018_keep['TimeAfterBootcamp'] = df_2018_keep.TimeAfterBootcamp.cat.add_categories('I got a job as a developer before completing the program')

### 3.32 - Languages Worked With
What programming languages has the respondent worked with extensively in the past year?  Multiple languages are allowed, which gives many unique variables.

In [170]:
getMissingPercent('LanguageWorkedWith')

LanguageWorkedWith - percent missing in 2017: 2.0365535248041775%
LanguageWorkedWith - percent missing in 2018: 2.0925842218617885%


In [171]:
df_2017_keep['LanguageWorkedWith']= df_2017_keep.LanguageWorkedWith.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['LanguageWorkedWith']= df_2018_keep.LanguageWorkedWith.cat.add_categories('NaN').fillna('NaN')

In [172]:
df_2017_keep['LanguageWorkedWith_Count'] = df_2017_keep['LanguageWorkedWith'].apply(get_count)
df_2018_keep['LanguageWorkedWith_Count'] = df_2018_keep['LanguageWorkedWith'].apply(get_count)

### 3.33 - Languages Want to Work With
Similar to section 3.32, but with languages the respondent would like to learn in the next year.

In [173]:
getMissingPercent('LanguageDesireNextYear')

LanguageDesireNextYear - percent missing in 2017: 9.138381201044385%
LanguageDesireNextYear - percent missing in 2018: 7.737509638650324%


In [174]:
df_2017_keep['LanguageDesireNextYear']= df_2017_keep.LanguageDesireNextYear.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['LanguageDesireNextYear']= df_2018_keep.LanguageDesireNextYear.cat.add_categories('NaN').fillna('NaN')

In [175]:
df_2017_keep['LanguageDesireNextYear_Count'] = df_2017_keep['LanguageDesireNextYear'].apply(get_count)
df_2018_keep['LanguageDesireNextYear_Count'] = df_2018_keep['LanguageDesireNextYear'].apply(get_count)

### 3.34 - Frameworks Worked With
Similar to section 3.32, but with frameworks (ex: Django, TensorFlow, Angular, etc)

In [176]:
getMissingPercent('FrameworkWorkedWith')

FrameworkWorkedWith - percent missing in 2017: 41.72323759791123%
FrameworkWorkedWith - percent missing in 2018: 30.862293599936187%


In [177]:
df_2017_keep['FrameworkWorkedWith']= df_2017_keep.FrameworkWorkedWith.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['FrameworkWorkedWith']= df_2018_keep.FrameworkWorkedWith.cat.add_categories('NaN').fillna('NaN')

In [178]:
df_2017_keep['FrameworkWorkedWith_Count'] = df_2017_keep['FrameworkWorkedWith'].apply(get_count)
df_2018_keep['FrameworkWorkedWith_Count'] = df_2018_keep['FrameworkWorkedWith'].apply(get_count)

### 3.35 - Frameworks Want to Work With
Similar to section 3.33 and 3.34, but with frameworks the respondent would like to learn next year.

In [179]:
getMissingPercent('FrameworkDesireNextYear')

FrameworkDesireNextYear - percent missing in 2017: 33.072236727589214%
FrameworkDesireNextYear - percent missing in 2018: 27.251455768566036%


In [180]:
df_2017_keep['FrameworkDesireNextYear']= df_2017_keep.FrameworkDesireNextYear.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['FrameworkDesireNextYear']= df_2018_keep.FrameworkDesireNextYear.cat.add_categories('NaN').fillna('NaN')

In [181]:
df_2017_keep['FrameworkDesireNextYear_Count'] = df_2017_keep['FrameworkDesireNextYear'].apply(get_count)
df_2018_keep['FrameworkDesireNextYear_Count'] = df_2018_keep['FrameworkDesireNextYear'].apply(get_count)

### 3.36 - Databases Worked With
Similar to section 3.32, but with databases (ex: Microsoft Azure, MySQL, MongoDB, etc.)

In [182]:
getMissingPercent('DatabaseWorkedWith')

DatabaseWorkedWith - percent missing in 2017: 18.085291557876417%
DatabaseWorkedWith - percent missing in 2018: 14.469940705682149%


In [183]:
df_2017_keep['DatabaseWorkedWith']= df_2017_keep.DatabaseWorkedWith.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['DatabaseWorkedWith']= df_2018_keep.DatabaseWorkedWith.cat.add_categories('NaN').fillna('NaN')

In [184]:
df_2017_keep['DatabaseWorkedWith_Count'] = df_2017_keep['DatabaseWorkedWith'].apply(get_count)
df_2018_keep['DatabaseWorkedWith_Count'] = df_2018_keep['DatabaseWorkedWith'].apply(get_count)

### 3.37 - Databases Want to Work With
Similar to section 3.33, but with databases.

In [185]:
getMissingPercent('DatabaseDesireNextYear')

DatabaseDesireNextYear - percent missing in 2017: 30.43516100957354%
DatabaseDesireNextYear - percent missing in 2018: 25.93262251056928%


In [186]:
df_2017_keep['DatabaseDesireNextYear']= df_2017_keep.DatabaseDesireNextYear.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['DatabaseDesireNextYear']= df_2018_keep.DatabaseDesireNextYear.cat.add_categories('NaN').fillna('NaN')

In [187]:
df_2017_keep['DatabaseDesireNextYear_Count'] = df_2017_keep['DatabaseDesireNextYear'].apply(get_count)
df_2018_keep['DatabaseDesireNextYear_Count'] = df_2018_keep['DatabaseDesireNextYear'].apply(get_count)

### 3.38 - Platforms Worked With
Similar to section 3.32 but with platforms (ex: Linux, Microsoft Azure, AWS, etc.)

In [188]:
getMissingPercent('PlatformWorkedWith')

PlatformWorkedWith - percent missing in 2017: 22.010443864229764%
PlatformWorkedWith - percent missing in 2018: 15.358026004413839%


In [189]:
df_2017_keep['PlatformWorkedWith']= df_2017_keep.PlatformWorkedWith.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['PlatformWorkedWith']= df_2018_keep.PlatformWorkedWith.cat.add_categories('NaN').fillna('NaN')

In [190]:
df_2017_keep['PlatformWorkedWith_Count'] = df_2017_keep['PlatformWorkedWith'].apply(get_count)
df_2018_keep['PlatformWorkedWith_Count'] = df_2018_keep['PlatformWorkedWith'].apply(get_count)

### 3.39 - Platforms Want to Work With
Similar to section 3.33, but with platforms.

In [191]:
getMissingPercent('PlatformDesireNextYear')

PlatformDesireNextYear - percent missing in 2017: 24.04699738903394%
PlatformDesireNextYear - percent missing in 2018: 20.904570714456646%


In [192]:
df_2017_keep['PlatformDesireNextYear']= df_2017_keep.PlatformDesireNextYear.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['PlatformDesireNextYear']= df_2018_keep.PlatformDesireNextYear.cat.add_categories('NaN').fillna('NaN')

In [193]:
df_2017_keep['PlatformDesireNextYear_Count'] = df_2017_keep['PlatformDesireNextYear'].apply(get_count)
df_2018_keep['PlatformDesireNextYear_Count'] = df_2018_keep['PlatformDesireNextYear'].apply(get_count)

### 3.40 - IDE
What development environment does the respondent use on a regular basis?  Examples include Sublime, RStudio, PyCharm, etc.  Multiple answers allowed.

In [194]:
getMissingPercent('IDE')

IDE - percent missing in 2017: 3.29852045256745%
IDE - percent missing in 2018: 3.1242521736818314%


In [195]:
df_2017_keep['IDE']= df_2017_keep.IDE.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['IDE']= df_2018_keep.IDE.cat.add_categories('NaN').fillna('NaN')

In [196]:
df_2017_keep['IDE_Count'] = df_2017_keep['IDE'].apply(get_count)
df_2018_keep['IDE_Count'] = df_2018_keep['IDE'].apply(get_count)

### 3.41 - Methodology
Asks the respondent what types of methodology they are familiar with.  Examples include pair programming, lean, and scrum.  Multiple answers allowed.

In [197]:
getMissingPercent('Methodology')

Methodology - percent missing in 2017: 35.50043516100958%
Methodology - percent missing in 2018: 16.323220505730013%


In [198]:
df_2017_keep['Methodology']= df_2017_keep.Methodology.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['Methodology']= df_2018_keep.Methodology.cat.add_categories('NaN').fillna('NaN')

In [199]:
df_2017_keep['Methodology_Count'] = df_2017_keep['Methodology'].apply(get_count)
df_2018_keep['Methodology_Count'] = df_2018_keep['Methodology'].apply(get_count)

### 3.42 - Version Control
Asks the respondent what version control (if at all) they use.  Multiple answers allowed.

In [200]:
getMissingPercent('VersionControl')

VersionControl - percent missing in 2017: 29.67798085291558%
VersionControl - percent missing in 2018: 3.6081788933499963%


In [201]:
df_2017_keep['VersionControl']= df_2017_keep.VersionControl.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['VersionControl']= df_2018_keep.VersionControl.cat.add_categories('NaN').fillna('NaN')

In [202]:
df_2017_keep['VersionControl_Count'] = df_2017_keep['VersionControl'].apply(get_count)
df_2018_keep['VersionControl_Count'] = df_2018_keep['VersionControl'].apply(get_count)

### 3.43 - Frequency of Checking in Code
Asks the respondent over the past year, how often they checked in or committed code.  Answers are similarly worded for the two years.  

In [203]:
getMissingPercent('CheckInCode')

CheckInCode - percent missing in 2017: 29.521322889469104%
CheckInCode - percent missing in 2018: 4.219734637985589%


In [204]:
list(df_2017_keep['CheckInCode'].unique())

['Multiple times a day',
 nan,
 'A few times a month',
 'Once a day',
 'A few times a week',
 'Never',
 'Just a few times over the year']

In [205]:
list(df_2018_keep['CheckInCode'].unique())

['A few times per week',
 'Weekly or a few times per month',
 'Multiple times per day',
 'Never',
 'Less than once per month',
 'Once a day',
 nan]

In [206]:
df_2017_keep['CheckInCode']= df_2017_keep.CheckInCode.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['CheckInCode']= df_2018_keep.CheckInCode.cat.add_categories('NaN').fillna('NaN')

In [207]:
checkInCode_dict = { 'Just a few times over the year': 'Less than once per month',  'A few times a month':  'Weekly or a few times per month',
                   'Multiple times a day':  'Multiple times per day',  'Once a day':  'Once a day',  'A few times a week': 'A few times per week',
                     'Never':  'Never', 'NaN': 'NaN'}

In [208]:
def convert_checkInCode(col):
    return checkInCode_dict[col]

In [209]:
df_2017_keep['CheckInCode'] = df_2017_keep['CheckInCode'].apply(convert_checkInCode)

### 3.44 - Stack Overflow Jobs
Respondents are asked if they have ever used or visited the Stack Overflow Jobs webpage.  It was difficult to combine responses between the two years, so I simplified answers to yes or no.

In [210]:
getMissingPercent('StackOverflowJobs')

StackOverflowJobs - percent missing in 2017: 7.6153176675369885%
StackOverflowJobs - percent missing in 2018: 2.6057592597516552%


In [211]:
list(df_2017_keep['StackOverflowJobs'].unique())

["Haven't done at all",
 'Several times',
 'Once or twice',
 nan,
 'At least once each week',
 'At least once each day']

In [212]:
list(df_2018_keep['StackOverflowJobs'].unique())

['Yes',
 'No, I knew that Stack Overflow had a jobs board but have never used or visited it',
 "No, I didn't know that Stack Overflow had a jobs board",
 nan]

In [213]:
df_2017_keep['StackOverflowJobs']= df_2017_keep.StackOverflowJobs.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['StackOverflowJobs']= df_2018_keep.StackOverflowJobs.cat.add_categories('NaN').fillna('NaN')

In [214]:
SOJobs_dict = {'Yes': 'Yes', 'No, I knew that Stack Overflow had a jobs board but have never used or visited it': 'No',
              "No, I didn't know that Stack Overflow had a jobs board": 'No', "Haven't done at all": 'No', 'Several times': 'Yes',
              'Once or twice': 'Yes', 'At least once each week': 'Yes', 'At least once each day': 'Yes', 'NaN': 'NaN'}

In [215]:
def convert_SOJobs(col):
    return SOJobs_dict[col]

In [216]:
df_2017_keep['StackOverflowJobs'] = df_2017_keep['StackOverflowJobs'].apply(convert_SOJobs)
df_2018_keep['StackOverflowJobs'] = df_2018_keep['StackOverflowJobs'].apply(convert_SOJobs)

### 3.45 -  Gender
Respondents are asked what gender(s) they identify with.  Multiple answers allowed, so there are more unique answers than just your typical male/female binary.

In [217]:
getMissingPercent('Gender')

Gender - percent missing in 2017: 5.230635335073977%
Gender - percent missing in 2018: 5.987928421388498%


In [218]:
df_2017_keep['Gender'].unique()

[Male, Female, NaN, Other, Female; Transgender, ..., Male; Female; Transgender, Transgender, Male; Gender non-conforming; Other, Female; Transgender; Other, Male; Transgender; Other]
Length: 20
Categories (19, object): [Male, Female, Other, Female; Transgender, ..., Transgender, Male; Gender non-conforming; Other, Female; Transgender; Other, Male; Transgender; Other]

In [219]:
df_2018_keep['Gender'].unique()

[Male, Female;Male;Transgender;Non-binary, genderquee..., Female;Male, Female, Male;Non-binary, genderqueer, or gender non-co..., ..., Female;Transgender;Non-binary, genderqueer, or..., Male;Transgender, Female;Male;Non-binary, genderqueer, or gender..., Male;Transgender;Non-binary, genderqueer, or g..., Female;Male;Transgender]
Length: 16
Categories (15, object): [Male, Female;Male;Transgender;Non-binary, genderquee..., Female;Male, Female, ..., Male;Transgender, Female;Male;Non-binary, genderqueer, or gender..., Male;Transgender;Non-binary, genderqueer, or g..., Female;Male;Transgender]

In [220]:
df_2017_keep['Gender']= df_2017_keep.Gender.cat.add_categories('I prefer not to answer').fillna('I prefer not to answer')
df_2018_keep['Gender']= df_2018_keep.Gender.cat.add_categories('I prefer not to answer').fillna('I prefer not to answer')

In [221]:
df_2017_keep['Gender_Count'] = df_2017_keep['Gender'].apply(get_count)
df_2018_keep['Gender_Count'] = df_2018_keep['Gender'].apply(get_count)

### 3.46 - Parents' Highest Education
Asks what is the respondent's' parents highest level of education.  Both years had similar answers but different wording.

In [222]:
getMissingPercent('EducationParents')

EducationParents - percent missing in 2017: 5.291557876414274%
EducationParents - percent missing in 2018: 8.085830519290594%


In [223]:
list(df_2017_keep['EducationParents'].unique())

['A professional degree',
 'High school',
 "A master's degree",
 "A bachelor's degree",
 'A doctoral degree',
 'Primary/elementary school',
 "Some college/university study, no bachelor's degree",
 nan,
 "I don't know/not sure",
 'No education',
 'I prefer not to answer']

In [224]:
list(df_2018_keep['EducationParents'].unique())

['Bachelor’s degree (BA, BS, B.Eng., etc.)',
 'Some college/university study without earning a degree',
 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
 'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
 'Associate degree',
 'They never completed any formal education',
 nan,
 'Other doctoral degree (Ph.D, Ed.D., etc.)',
 'Professional degree (JD, MD, etc.)',
 'Primary/elementary school']

In [225]:
df_2017_keep['EducationParents']= df_2017_keep.EducationParents.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['EducationParents']= df_2018_keep.EducationParents.cat.add_categories('NaN').fillna('NaN')

In [226]:
df_2017_keep['EducationParents']= df_2017_keep.EducationParents.cat.add_categories('Associate degree')
df_2018_keep['EducationParents']= df_2018_keep.EducationParents.cat.add_categories(['I don\'t know/not sure', 'I prefer not to answer'])

In [227]:
educationParents_dict = {'NaN': 'NaN', 'A professional degree': 'A professional degree',  
                         'Professional degree (JD, MD, etc.)':'A professional degree',
                         "A bachelor's degree":  "A bachelor's degree",
                        'Bachelor’s degree (BA, BS, B.Eng., etc.)': "A bachelor's degree",
                         "A master's degree":  "A master's degree",
                         'Master’s degree (MA, MS, M.Eng., MBA, etc.)': "A master's degree",
                         'High school':  'High school',
                         'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 'High school',
                         'A doctoral degree':  'A doctoral degree',
                         'Other doctoral degree (Ph.D, Ed.D., etc.)': 'A doctoral degree',
                        'Some college/university study, no bachelor\'s degree': 'Some college/university',
                         'Some college/university study without earning a degree': 'Some college/university',
                         'Primary/elementary school': 'Primary/elementary school',
                         'No education':  'No education',  'They never completed any formal education': 'No education',
                         'I prefer not to answer':  'I prefer not to answer',  "I don't know/not sure":  "I don't know/not sure",
                         'Associate degree':  'Associate degree'}

In [228]:
def convert_educationParents(col):
    return educationParents_dict[col]

In [229]:
df_2017_keep['EducationParents'] = df_2017_keep['EducationParents'].apply(convert_educationParents)
df_2018_keep['EducationParents'] = df_2018_keep['EducationParents'].apply(convert_educationParents)

### 3.47 - Race
Respondents are asked what race(s) they identify with.  Multiple answers allowed.

In [230]:
getMissingPercent('Race')

Race - percent missing in 2017: 8.485639686684072%
Race - percent missing in 2018: 12.438512058283921%


In [231]:
df_2017_keep['Race']= df_2017_keep.Race.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['Race']= df_2018_keep.Race.cat.add_categories('NaN').fillna('NaN')

In [232]:
df_2017_keep['Race_Count'] = df_2017_keep['Race'].apply(get_count)
df_2018_keep['Race_Count'] = df_2018_keep['Race'].apply(get_count)

### 3.48 - Survey too Long
Lastly, respondents are asked if the survey was too long.  

In [233]:
getMissingPercent('SurveyLong')

SurveyLong - percent missing in 2017: 5.97911227154047%
SurveyLong - percent missing in 2018: 4.804701002419633%


In [234]:
list(df_2017_keep['SurveyLong'].unique())

['Somewhat agree',
 'Disagree',
 'Strongly agree',
 'Agree',
 'Strongly disagree',
 nan]

In [235]:
list(df_2018_keep['SurveyLong'].unique())

['The survey was an appropriate length',
 'The survey was too long',
 nan,
 'The survey was too short']

In [236]:
surveyLength_dict = {'Strongly disagree':  'The survey was too short', 'Disagree': 'The survey was too short',
                    'Somewhat agree': 'The survey was an appropriate length',  'Agree':  'The survey was too long',
                     'Strongly agree': 'The survey was too long', 'NaN': 'NaN'}


In [237]:
def convert_survey(col):
    return surveyLength_dict[col]

In [238]:
df_2017_keep['SurveyLong']= df_2017_keep.SurveyLong.cat.add_categories('NaN').fillna('NaN')
df_2018_keep['SurveyLong']= df_2018_keep.SurveyLong.cat.add_categories('NaN').fillna('NaN')

In [239]:
df_2017_keep['SurveyLong'] = df_2017_keep['SurveyLong'].apply(convert_survey)

# 4 - Saving the Data
Before saving the data, lets check the data types for each column is what we want.

### 4.1 - Checking Datatypes

In [240]:
df_2017_keep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11490 entries, 2 to 51390
Data columns (total 64 columns):
ID                               11490 non-null float32
Hobby                            11490 non-null object
Country                          11490 non-null category
Student                          11490 non-null category
Employment                       11490 non-null category
Education                        11490 non-null category
UndergradMajor                   11490 non-null category
CompanySize                      11490 non-null category
YearsCoding                      11490 non-null object
YearsCodingProf                  11490 non-null object
DevType                          11490 non-null category
CareerSatisfaction               11490 non-null object
JobSatisfaction                  11490 non-null object
KinshipDevelopers                11490 non-null float64
CompetePeers                     11490 non-null float64
LastNewJob                       11490 non-null c

In [241]:
df_2018_keep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37609 entries, 1 to 89960
Data columns (total 64 columns):
ID                               37609 non-null float32
Hobby                            37609 non-null category
Country                          37609 non-null category
Student                          37609 non-null category
Employment                       37609 non-null category
Education                        37609 non-null category
UndergradMajor                   37609 non-null category
CompanySize                      37609 non-null category
DevType                          37609 non-null category
YearsCoding                      37609 non-null object
YearsCodingProf                  37609 non-null object
JobSatisfaction                  37609 non-null category
CareerSatisfaction               37609 non-null category
LastNewJob                       37609 non-null object
AssessJobIndustry                37609 non-null object
AssessJobFinances                37609 non-nu

I see lots of object categories, and I'm fairly certain the 'Count' columns don't need to be 64 bit.  Lets downcast both dataframes again.

In [242]:
downcast(df_2017_keep)
downcast(df_2018_keep)

In [243]:
get_memoryUsage(df_2017_keep)

Total memory usage for category: 9.28803 MB
Total memory usage for float32: 0.57549 MB
Total memory usage for int8: 0.56453 MB

Total Memory Usage: 9.63 MB


In [244]:
get_memoryUsage(df_2018_keep)

Total memory usage for category: 35.69232 MB
Total memory usage for float32: 0.86080 MB
Total memory usage for int8: 0.82493 MB

Total Memory Usage: 36.80 MB


As expected, objects were converted to categories, and the 'count' columns were converted to int8.

### 4.2 - Saving to Feather
Finally, we can save the updated columns to feather format and it should be able to run through a random forest model without a problem.  Since I deleted some rows, the index is not in sequential order.  The index must be reset before saving to feather.

In [245]:
df_2017_keep.reset_index(inplace = True)
df_2018_keep.reset_index(inplace = True)

In [246]:
df_2017_keep.to_feather('tmp/df_2017_2keep')
df_2018_keep.to_feather('tmp/df_2018_2keep')

In [247]:
df_2017_keep.columns

Index(['index', 'ID', 'Hobby', 'Country', 'Student', 'Employment', 'Education',
       'UndergradMajor', 'CompanySize', 'YearsCoding', 'YearsCodingProf',
       'DevType', 'CareerSatisfaction', 'JobSatisfaction', 'KinshipDevelopers',
       'CompetePeers', 'LastNewJob', 'AssessJobIndustry', 'AssessJobDept',
       'AssessJobTech', 'AssessJobCompensation', 'AssessJobOffice',
       'AssessJobRemote', 'AssessJobProfDevel', 'AssessJobDiversity',
       'AssessJobProduct', 'AssessJobFinances', 'UpdateCV', 'Currency',
       'EducationTypes', 'SelfTaughtTypes', 'TimeAfterBootcamp',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'FrameworkWorkedWith',
       'FrameworkDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'IDE', 'Methodology', 'VersionControl',
       'CheckInCode', 'StackOverflowJobs', 'Gender', 'EducationParents',
       'Race', 'SurveyLong', 'Salary', 'DevType_Count', 'EducationTypes_Count',
 

### test data

In [248]:
df_2017_keep['DevType'][:5]

0                                                Other
1              Embedded applications/devices developer
2    Web developer; Embedded applications/devices d...
3                                                Other
4                                        Web developer
Name: DevType, dtype: category
Categories (1824, object): [Data scientist, Data scientist; Database administrator, Data scientist; Database administrator; DevOps..., Data scientist; Database administrator; Other, ..., Web developer; Systems administrator; DevOps s..., Web developer; Systems administrator; Other, Web developer; Systems administrator; Quality ..., NaN]

In [249]:
test = pd.DataFrame({'A':['adlkfslkfd', 'Nan', 'NaN', 'joke;asdlfk;asdf', 'adsf;dsf;asdf;dsa;fds;;fd;faf;ds'], 
                     'B': [np.nan, 'No', 'Yes, fdas', 'Yes', 'No'], 'C':[45, 65,23,45,74]})

In [250]:
test

,A,B,C
0,adlkfslkfd,NaN,45
1,Nan,No,65
2,NaN,"Yes, fdas",23
3,joke;asdlfk;asdf,Yes,45
4,adsf;dsf;asdf;dsa;fds;;fd;faf;ds,No,74


In [251]:
test['A'].apply(get_count)

0    1
1    1
2    0
3    3
4    9
Name: A, dtype: int64

In [252]:
def test_func(col):
    return len(col.split(';'))
#     col_suffix = '_count'
#     for row in df[col]:
#         df[col + col_suffix] = row.split(';')

In [253]:
test['A_Count'] = test['A'].apply(test_func)

In [254]:
test

,A,B,C,A_Count
0,adlkfslkfd,NaN,45,1
1,Nan,No,65,1
2,NaN,"Yes, fdas",23,1
3,joke;asdlfk;asdf,Yes,45,3
4,adsf;dsf;asdf;dsa;fds;;fd;faf;ds,No,74,9


In [255]:
len('web;asdf'.split(';'))

2

In [256]:
df_2018[df_2018['Respondent']==21]

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,Salary,SalaryType,ConvertedSalary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,IDE,OperatingSystem,NumberMonitors,Methodology,VersionControl,CheckInCode,AdBlocker,AdBlockerDisable,AdBlockerReasons,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
14,21.0,No,No,Netherlands,"Yes, full-time",Employed full-time,"Secondary school (e.g. American high school, G...",NaN,20 to 99 employees,Back-end developer;Front-end developer;Student,0-2 years,0-2 years,Neither satisfied nor dissatisfied,Moderately dissatisfied,Working in a career completely unrelated to so...,"I’m not actively looking, but I am open to new...",Between 2 and 4 years ago,1.0,9.0,2.0,10.0,6.0,3.0,4.0,5.0,7.0,8.0,2.0,3.0,6.0,11.0,9.0,8.0,10.0,4.0,5.0,7.0,1.0,5.0,3.0,2.0,1.0,4.0,1.0,2.0,6.0,7.0,3.0,4.0,5.0,My job status or other personal status changed,Euros (€),0,Monthly,0.0,EUR,NaN,Less than a month,Received on-the-job training in software devel...,NaN,NaN,NaN,Disagree,Strongly disagree,Strongly agree,Java;JavaScript;PHP;VB.NET;HTML;CSS,NaN,SQL Server;MySQL,NaN,Windows Desktop or Server,NaN,NaN,NaN,Notepad++;Visual Studio,Windows,2,Agile;Scrum,Copying and pasting files to network shares,Never,No,NaN,NaN,Strongly agree,Somewhat agree,Neither agree nor disagree,Clicked on an online advertisement,4.0,5.0,6.0,2.0,1.0,7.0,3.0,Artificial intelligence surpassing human intel...,NaN,The developers or the people creating the AI,I'm worried about the dangers more than I'm ex...,No,"Yes, and publicly",The person who came up with the idea,Yes,8,Daily or almost daily,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, and I don't know what that is",NaN,No,Not at all interested,Not at all interested,Not at all interested,Not at all interested,Not at all interested,Between 6:01 - 7:00 AM,9 - 12 hours,1 - 2 hours,Never,NaN,Daily or almost every day,Male,NaN,Associate degree,White or of European descent,18 - 24 years old,No,NaN,The survey was an appropriate length,Neither easy nor difficult
